# Data storage and project layout

- The initial dataset should be stored as `mol2` or `mol2.gz` files in a directory. For example:

In [8]:
!ls -lh 80698/mol2/

total 2343768
-rw-r--r--  1 sebastian  staff   572M May  6 21:20 1.mol2
-rw-r--r--  1 sebastian  staff   572M May  6 21:22 2.mol2


- for a new project, it's best to create a new directory, for instance, we create a new subdirectory in 80698:

In [2]:
!mkdir 80698/proj-1/

Using the `count_mol2.py` script, we can count the number of structures in each mol2 file in the input directory:

In [9]:
!python ../../../tools/count_mol2.py -i 80698/mol2

1.mol2 : 135966
2.mol2 : 136144
Total : 272110


Optionally, you may have files with data about the molecules, for instance:

In [ ]:
!ls -lh 80698/csv/

# General Blacklist & Whitelist filtering

- `mol2_to_id.py` generates a list of molecule IDs from MOL2 files
- `id_to_mol2.py` filters mol2 files by IDs and creates new mol2 files. Via whitelisting, molecules that match those IDs are written. Via blacklisting, all molecules but the molecules that are in the list are written

In [11]:
!python ../../../tools/mol2_to_id.py -i 80698/mol2 -o 80698/proj-1/all-mol2ids.txt

Processing 80698/mol2/1.mol2
Processing 80698/mol2/2.mol2
Finished


In [12]:
!head 80698/proj-1/all-mol2ids.txt

ZINC57271411
ZINC50764925
ZINC65255333
ZINC06394508
ZINC65292537
ZINC65375610
ZINC31820077
ZINC65395084
ZINC00205726
ZINC01458151


- Say we are interested in a subset of molecules only. Consider this example: 1st we create a list of IDs:

In [7]:
%%writefile ./80698/proj-1/selected-example-mol2ids.txt
ZINC57271411
ZINC06394508
ZINC65292537
ZINC31820077
ZINC01458151

Overwriting ./80698/proj-1/selected-example-mol2ids.txt


### Whitelisting

In [23]:
!python ../../../tools/id_to_mol2.py \
--input 80698/mol2 \
--output 80698/proj-1/selected-example-mol2ids \
--id_file ./80698/proj-1/selected-example-mol2ids.txt \
--whitelist True

Processing 80698/mol2/1.mol2
Processing 80698/mol2/2.mol2
Finished


Now, the output directory, `80698/proj-1/selected-example-mol2ids`, should contain only mol2 files with the selected IDs:

In [24]:
!ls 80698/proj-1/selected-example-mol2ids

1.mol2 2.mol2


In [25]:
!grep 'ZINC' 80698/proj-1/selected-example-mol2ids/1.mol2

ZINC57271411
ZINC06394508
ZINC65292537
ZINC31820077
ZINC01458151


In [28]:
!grep -C 1 'ZINC' 80698/proj-1/selected-example-mol2ids/1.mol2 

@<TRIPOS>MOLECULE
ZINC57271411
   50    52     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC06394508
   49    52     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC65292537
   47    50     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC31820077
   48    50     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC01458151
   25    25     0     0     0


### Blacklisting

Similar to the previous approach, using a whitelist filter, we can do blacklist filtering, which means that all molecules are selected but the ones contained in the ID files. Set whitelist to False.

In [46]:
!python ../../../tools/id_to_mol2.py \
--input 80698/mol2 \
--output 80698/proj-1/nonselected-example-mol2ids \
--id_file ./80698/proj-1/selected-example-mol2ids.txt \
--whitelist false

Processing 80698/mol2/1.mol2
Processing 80698/mol2/2.mol2
Finished


In [47]:
!python ../../../tools/count_mol2.py -i 80698/proj-1/nonselected-example-mol2ids/

1.mol2 : 135961
2.mol2 : 136144
Total : 272105


# First Filtering Step -- Filtering via Features from Data Tables

In [48]:
!head 80698/csv/3_prop.xls

ZINC_ID	MWT	LogP	Desolv_apolar	Desolv_polar	HBD	HBA	tPSA	Charge	NRB	SMILES
ZINC00000010	217.2	1.42	5.57	-41.98	0	4	66	-1	2	C[C@@]1(C(=O)C=C(O1)C(=O)[O-])c2ccccc2
ZINC00000012	289.356	1.28	4.89	-24.55	2	4	66	0	5	c1ccc(cc1)C(c2ccccc2)[S@](=O)CC(=O)NO
ZINC00000017	281.337	1.33	3.06	-23.33	2	6	87	0	4	CCC[S@](=O)c1ccc2c(c1)[nH]/c(=N\C(=O)OC)/[nH]2
ZINC00000017	281.337	1.33	3.07	-19.2	2	6	87	0	4	CCC[S@](=O)c1ccc2c(c1)[nH]/c(=N/C(=O)OC)/[nH]2
ZINC00000018	212.318	2.00	5.87	-8.2	1	3	32	0	4	CC(C)C[C@@H]1C(=O)N(C(=S)N1)CC=C
ZINC00000021	288.411	3.85	4.02	-40.52	1	3	30	1	6	CCC(=O)O[C@]1(CC[NH+](C[C@@H]1CC=C)C)c2ccccc2
ZINC00000022	218.276	3.21	0.47	-48.57	1	3	52	-1	5	C[C@@H](c1ccc(cc1)NCC(=C)C)C(=O)[O-]
ZINC00000025	251.353	3.60	2.4	-41.56	2	2	40	1	5	C[C@H](Cc1ccccc1)[NH2+][C@@H](C#N)c2ccccc2
ZINC00000030	297.422	2.94	0.89	-37.97	3	3	47	1	6	C[C@@H](CC(c1ccccc1)(c2ccccc2)C(=O)N)[NH+](C)C


- A valid query looks like this:
 
- Correct: `"(MWT >= 200) & (NRB <= 7)"`
- Wrong: `"( MWT >= 200) & ( NRB <= 7)"` [spacing between parentheses and column names]
- Wrong: `"MWT >= 200 & NRB <= 7"` [expressions seperated by logical '&' operator not enclosed in parentheses]
- Wrong: `"(mwt >= 200) & (nrb <= 7)"` [column names don't match the columns in the data table file]
- Wrong: `"(mwt>=200) & (nrb<=7)"` [no whitespace before and after operators for comparison]


In [107]:
!python ../../../tools/datatable_to_id.py \
--input 80698/csv/3_prop.xls \
--output 80698/proj-1/selected-mol2ids.txt \
--id_column ZINC_ID \
--selection "(NRB <= 7) & (MWT >= 200)"

Using columns: ['ZINC_ID', 'NRB', 'MWT']
Using selection: (chunk.NRB <= 7) & (chunk.MWT >= 200)
Processed: 18000000
Selected: 17599186


Next, we are going to use these IDs to select molecules from the existing mol2 files:

In [108]:
!python ../../../tools/id_to_mol2.py \
--input 80698/mol2 \
--output 80698/proj-1/selected-mol2ids \
--id_file ./80698/proj-1/selected-mol2ids.txt \
--whitelist True

Processing 80698/mol2/1.mol2
Processing 80698/mol2/2.mol2
Finished


In [109]:
!python ../../../tools/count_mol2.py -i 80698/proj-1/selected-mol2ids/

1.mol2 : 133655
2.mol2 : 133607
Total : 267262


# Second Filtering Step -- Presence and absence of functional groups

- allowed columns
  - atom_id
  - atom_name
  - atom_type
  - subst_id
  - subst_name
  - charge
  
(may change with updated biopandas)

a) simultaneously true (here impossible): "((atom_type == 'S.3') | (atom_type == 'S.o2')) & (atom_type == 'O.2')"

- tries to find an S.3 or S.o2 atom that is also an O.2 atom at the same time

b) simultaneously true and valid: "((atom_type == 'S.3') | (atom_type == 'S.o2')) & (charge < 0.0)"

- tries to find an S.3 or S.o2 atom that has a negative charge

c) otherwise, "((atom_type == 'S.3') | (atom_type == 'S.o2')) --> (atom_type == 'O.2')"

- tries to find an S.3 or S.o2 atom, then tries to find an O.2 atom as well

d) when in doubt, one can run (as alternative for c) the tool 2 times. 1 time to select "(atom_type == 'S.3') | (atom_type == 'S.o2')" and a second time to select "(atom_type == 'O.2')".

In [134]:
import time

start = time.time()

In [146]:
!python ../../../tools/funcgroup_to_id.py \
--input 80698/proj-1/selected-mol2ids/ \
--output 80698/proj-1/3keto-and-sulfur-mol2ids.txt \
--selection "((atom_type == 'S.3') | (atom_type == 'S.o2')) & (atom_type == 'O.2')"

Processing 80698/proj-1/selected-mol2ids/1.mol2
((pdmol.df.atom_type == 'S.3') | (pdmol.df.atom_type == 'S.o2')) & (pdmol.df.atom_type == 'O.2')


In [136]:
time.time() - start

1783.5474519729614

In [147]:
pdmol = PandasMol2()
for mol2 in split_multimol2('80698/proj-1/selected-mol2ids/1.mol2'):
    pdmol.read_mol2_from_list(mol2_lines=mol2[1],
                              mol2_code=mol2[0])
    break

In [155]:
pd.eval("((pdmol.df.atom_name == 'C1') | (pdmol.df.atom_name == 'C2')) | (pdmol.df.atom_name == 'C3')")

0      True
1      True
2      True
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
Name: atom_name, dtype: bool

In [ ]:
"((pdmol.df.atom_name == 'C1') | (pdmol.df.atom_type == 'S.o2')) & (pdmol.df.atom_type == 'O.2')"

In [148]:
pdmol.df

,atom_id,atom_name,x,y,z,atom_type,subst_id,subst_name,charge
0,1,C1,-0.0444,1.4896,-0.0641,C.3,1,<0>,-0.1205
1,2,C2,-0.0072,-0.0167,-0.0886,C.ar,1,<0>,-0.0825
2,3,C3,0.0336,-0.6853,-1.2981,C.ar,1,<0>,-0.1091
3,4,C4,0.0678,-2.0643,-1.3286,C.ar,1,<0>,-0.0938
4,5,C5,0.0617,-2.7892,-0.1354,C.ar,1,<0>,-0.1256
5,6,C6,0.0210,-2.1148,1.0846,C.ar,1,<0>,-0.0557
6,7,C7,-0.0083,-0.7288,1.1045,C.ar,1,<0>,0.1325
7,8,N1,-0.0438,-0.0471,2.3263,N.am,1,<0>,-0.6738
8,9,C8,-0.7192,-0.5687,3.3695,C.2,1,<0>,0.5077
9,10,O1,-1.2225,-1.6684,3.2774,O.2,1,<0>,-0.5182


In [141]:
import time

start = time.time()

In [142]:
!python  /Users/sebastian/Desktop/screenlamp_no_db/02_get_keto_sulfate_mp.py \
--input_dir 80698/proj-1/selected-mol2ids/ \
--output_dir /Users/sebastian/Desktop/temp \
--processes 1

Processing 80698/proj-1/selected-mol2ids/1.mol2
Processing 80698/proj-1/selected-mol2ids/2.mol2


In [143]:
time.time() - start

1394.1292309761047

- compare speed with previous (old) script!!
- add multiprocessing to the above!!!